In [516]:
import pandas as pd
import numpy as np
import os, sys
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
from openpyxl.styles import Border, Side

# Exercise 1: Data Analysis Automation

In [517]:
# Load the provided Sample-sales_data.xlsx file into a pandas DataFrame
df = pd.read_excel(r'C:\Repositories\DI-Bootcamp\Week14\Day6\ExercisesXPGold\Sample-Sales-Data.xlsx')
display(df.shape)
display(df.head())

(390, 5)

,Postcode,Sales_Rep_ID,Sales_Rep_Name,Year,Value
0,2121,456,Jane,2011,84219.497311
1,2092,789,Ashish,2012,28322.192268
2,2128,456,Jane,2013,81878.997241
3,2073,123,John,2011,44491.142121
4,2134,789,Ashish,2012,71837.720959


In [518]:
# Check for nulls and show datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Postcode        390 non-null    int64  
 1   Sales_Rep_ID    390 non-null    int64  
 2   Sales_Rep_Name  390 non-null    object 
 3   Year            390 non-null    int64  
 4   Value           390 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 15.4+ KB


In [519]:
# Clean Non Numerical Columns
df['Sales_Rep_Name'] = df['Sales_Rep_Name'].str.strip().str.title()

In [520]:
# Export the cleaned and modified DataFrame to a new Excel file named cleaned_sales_data.xlsx.
with pd.ExcelWriter("cleaned_sales_data.xlsx", engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Holiday Data', index=False)

# Exercise 2: Automating Excel Reports

In [521]:
# Load Monthly_Sales Data to Pandas Dataframe
file_path = r'C:\Repositories\DI-Bootcamp\Week14\Day6\ExercisesXPGold\monthly-sales.xlsx'
df = pd.read_excel(file_path)

# Check
df.head()

,Month,Monthly Sales
0,2008-01,154000
1,2008-02,96000
2,2008-03,73000
3,2008-04,51000
4,2008-05,53000


In [522]:
# Group the data by month and calculate the total sales and the number of transactions for each month
# 1. Separate Month and Year into Columns (Drop Original Month Column after)
df[['year', 'month']] = df['Month'].str.split('-', expand=True)
df = df.drop(columns='Month')

# 2. Calculate Total Sales and Transactions for Each Month (using groupby)
df_grouped = df.groupby('month')['Monthly Sales'].agg(Total_Sales=('sum'))

# Write this summary to a new Excel sheet named Monthly Summary in the same workbook.
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_grouped.to_excel(writer, sheet_name='Monthly Summary', index=True)

In [523]:
# Format the header row to be bold and apply a background color for better readability.
# First Load the Workbook
wb = load_workbook(file_path)

# Navigate to the Correct Worksheet
ws = wb['Monthly Summary']

# Format Header Row (Bold + Color)
header_font = Font(bold=True, color='FFFFFF')  # Bold white font
header_fill = PatternFill(start_color='4F81BD', end_color='4F81BD', fill_type='solid')  # Blue background

# Loop Through Each Cell in the First Row (Header Row) of the Sheet
for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font

# Save the updated Excel workbook as monthly_sales_report.xlsx
wb.save('monthly_sales_report.xlsx')

# Exercise 3: Advanced Workbook Manipulation

In [547]:
sales = [
    ['Product ID', 'Product Name', 'Quantity Sold', 'Total Revenue'],
    [101, 'Widget A', 150, 3000],
    [102, 'Widget B', 200, 5000],
    [103, 'Widget C', 120, 3600],
    [104, 'Widget D', 80, 2400]
]

inventory = [
    ['Product ID', 'Product Name', 'Stock Quantity', 'Reorder Level'],
    [101, 'Widget A', 300, 50],
    [102, 'Widget B', 150, 30],
    [103, 'Widget C', 200, 40],
    [104, 'Widget D', 100, 20]
]

customers = [
    ['Customer ID', 'Customer Name', 'Contact Information'],
    [1, 'John Smith', 'john.smith@example.com'],
    [2, 'Jane Doe', 'jane.doe@example.com'],
    [3, 'Mike Johnson', 'mike.j@example.com'],
    [4, 'Emily Brown', 'emily.brown@example.com']
]

In [ ]:
# Create a new Excel workbook
wb = Workbook()

# Create three sheets named Sales, Inventory, and Customers.
# Sheet 1
ws1 = wb.active
ws1.title = 'Sales'

# Sheet 2
ws2 = wb.create_sheet(title="Inventory")

# Sheet 3
ws3 = wb.create_sheet(title="Customers")

sheets = [ws1, ws2, ws3]
sets = [sales, inventory, customers]
for i, sheet in enumerate(sheets):
    # Add Each 'Row' in Dataset to the Sheet
    for row in sets[i]:
        sheet.append(row)
    
    # Make Header Row Bold
    for cell in sheet[1]:
        cell.font = Font(bold=True)
    
    # Adjust Column Width
    for column in sheet.columns:
        max_length = max(len(str(cell.value)) if cell.value is not None else 0 for cell in column)
        adjusted_width = max_length + 2
        sheet.column_dimensions[get_column_letter(column[0].column)].width = adjusted_width
    
    # Add borders
    borders = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin'))

    # Loop through all cells in the table range
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        for cell in row:
            cell.border = borders

# Save Worksheet
wb.save("business_data.xlsx")

# Exercise 4: Data Visualization in Excel

In [530]:
# Load file as a pandas dataframe
file_path = r'C:\Repositories\DI-Bootcamp\Week14\Day6\ExercisesXPGold\Excel - Sales Performance Dashboard.xlsx'
df = pd.read_excel(file_path)

# Check
display(df.head())
display(df.info())

,Order ID,Order Date,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,CA-2017-152156,2017-11-08,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,CA-2017-152156,2017-11-08,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,CA-2017-138688,2017-06-12,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,US-2016-108966,2016-10-11,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,US-2016-108966,2016-10-11,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       9800 non-null   object        
 1   Order Date     9800 non-null   datetime64[ns]
 2   Customer ID    9800 non-null   object        
 3   Customer Name  9800 non-null   object        
 4   Segment        9800 non-null   object        
 5   Country        9800 non-null   object        
 6   City           9800 non-null   object        
 7   State          9800 non-null   object        
 8   Postal Code    9800 non-null   int64         
 9   Region         9800 non-null   object        
 10  Product ID     9800 non-null   object        
 11  Category       9800 non-null   object        
 12  Sub-Category   9800 non-null   object        
 13  Product Name   9800 non-null   object        
 14  Sales          9800 non-null   float64       
dtypes: datetime64[ns](1),

None

pd.**pivot_table**(data,
               values='ColumnToAggregate',
               index='RowGroupBy',
               columns='ColumnGroupBy',
               aggfunc='function')

In [531]:
# Insert the data into an Excel sheet named Performance
# Create New WorkBook
wb = Workbook()
ws1 = wb.active

# Convert Dataframe to Pivot_Table
pivot = pd.pivot_table(df, values='Sales', index='Region', aggfunc='sum')

with pd.ExcelWriter("sales_performance_chart.xlsx", engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Data', index=False)
    pivot.to_excel(writer, sheet_name='Performance')

In [ ]:
from openpyxl import load_workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.chart.label import DataLabelList

# Load workbook and sheet
file_path = r'C:\Repositories\DI-Bootcamp\Week14\Day6\ExercisesXPGold\sales_performance_chart.xlsx'
wb = load_workbook(file_path)
ws = wb['Performance']

# Detect columns dynamically
headers = [cell.value for cell in ws[1]]
region_col = headers.index("Region") + 1
sales_col = headers.index("Sales") + 1

# Build references
cats = Reference(ws, min_col=region_col, min_row=2, max_row=ws.max_row) 
values = Reference(ws, min_col=sales_col, min_row=1, max_row=ws.max_row)

# Create chart
chart = BarChart()
chart.type = "col" # Vertical Bars ('bar' = horizontal)
chart.title = "Sales per Region"
chart.y_axis.title = "Sales"
chart.x_axis.title = "Region"

# Choose Data for Chart
chart.add_data(values, titles_from_data=True)
chart.set_categories(cats)

# Style (optional)
chart.width = 14
chart.height = 8
chart.dataLabels = DataLabelList()
chart.dataLabels.showVal = True
chart.dataLabels.showCatName = False # hide category names
chart.dataLabels.showSerName = False  

# Add chart to sheet
ws.add_chart(chart, "E2")

# Save updated workbook
wb.save(file_path)

# Exercise 5: Using Scenario Manager for Strategic Planning

In [545]:
budget_data = [
    {"Category": "Marketing", "Subcategory": "Digital Advertising", "Monthly Budget ($)": 10000, "Quarterly Budget ($)": 30000, "Annual Budget ($)": 120000},
    {"Category": "Marketing", "Subcategory": "Print Media", "Monthly Budget ($)": 5000, "Quarterly Budget ($)": 15000, "Annual Budget ($)": 60000},
    {"Category": "Marketing", "Subcategory": "Events & Sponsorships", "Monthly Budget ($)": 8000, "Quarterly Budget ($)": 24000, "Annual Budget ($)": 96000},
    {"Category": "Marketing", "Subcategory": "Market Research", "Monthly Budget ($)": 3000, "Quarterly Budget ($)": 9000, "Annual Budget ($)": 36000},
    {"Category": "R&D", "Subcategory": "Product Development", "Monthly Budget ($)": 20000, "Quarterly Budget ($)": 60000, "Annual Budget ($)": 240000},
    {"Category": "R&D", "Subcategory": "Innovation Programs", "Monthly Budget ($)": 15000, "Quarterly Budget ($)": 45000, "Annual Budget ($)": 180000},
    {"Category": "R&D", "Subcategory": "Testing & Prototyping", "Monthly Budget ($)": 10000, "Quarterly Budget ($)": 30000, "Annual Budget ($)": 120000},
    {"Category": "Operations", "Subcategory": "Staff Salaries", "Monthly Budget ($)": 50000, "Quarterly Budget ($)": 150000, "Annual Budget ($)": 600000},
    {"Category": "Operations", "Subcategory": "Office Supplies", "Monthly Budget ($)": 2000, "Quarterly Budget ($)": 6000, "Annual Budget ($)": 24000},
    {"Category": "Operations", "Subcategory": "Utilities", "Monthly Budget ($)": 3500, "Quarterly Budget ($)": 10500, "Annual Budget ($)": 42000},
    {"Category": "Operations", "Subcategory": "Maintenance", "Monthly Budget ($)": 4000, "Quarterly Budget ($)": 12000, "Annual Budget ($)": 48000},
    {"Category": "Sales", "Subcategory": "Sales Team Commissions", "Monthly Budget ($)": 12000, "Quarterly Budget ($)": 36000, "Annual Budget ($)": 144000},
    {"Category": "Sales", "Subcategory": "Client Entertainment", "Monthly Budget ($)": 6000, "Quarterly Budget ($)": 18000, "Annual Budget ($)": 72000},
    {"Category": "Sales", "Subcategory": "Sales Training", "Monthly Budget ($)": 4500, "Quarterly Budget ($)": 13500, "Annual Budget ($)": 54000},
    {"Category": "IT", "Subcategory": "Software Licenses", "Monthly Budget ($)": 5500, "Quarterly Budget ($)": 16500, "Annual Budget ($)": 66000},
    {"Category": "IT", "Subcategory": "Hardware Maintenance", "Monthly Budget ($)": 3000, "Quarterly Budget ($)": 9000, "Annual Budget ($)": 36000},
    {"Category": "IT", "Subcategory": "IT Support Services", "Monthly Budget ($)": 7000, "Quarterly Budget ($)": 21000, "Annual Budget ($)": 84000},
    {"Category": "Human Resources", "Subcategory": "Recruitment", "Monthly Budget ($)": 4000, "Quarterly Budget ($)": 12000, "Annual Budget ($)": 48000},
    {"Category": "Human Resources", "Subcategory": "Employee Benefits", "Monthly Budget ($)": 18000, "Quarterly Budget ($)": 54000, "Annual Budget ($)": 216000},
    {"Category": "Human Resources", "Subcategory": "Training & Development", "Monthly Budget ($)": 6500, "Quarterly Budget ($)": 19500, "Annual Budget ($)": 78000}
]